In [147]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import warnings
warnings.filterwarnings("ignore")

In [148]:
prices = pd.read_csv('prices.csv', index_col='Date', parse_dates=True)
volumechanges = pd.read_csv('Volume.csv', index_col='Date', parse_dates=True).pct_change()*100


In [149]:
volumechanges

,MSFT,AAPL,AMZN,FB,JPM,GOOGL,GOOG,JNJ,V,PG,...,JWN,LB,UAA,GPS,UA,COTY,NWS,ORCL,WCG,ARNC
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,1.673167,-4.370911,8.498784,NaN,-2.461062,NaN,NaN,68.494744,NaN,-0.098246,...,-21.015643,24.566684,NaN,-36.837662,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,18.368041,51.903908,119.914910,NaN,-25.665762,NaN,NaN,5.208253,NaN,19.378102,...,-20.177956,-63.722713,NaN,81.723174,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,-14.178983,-1.329630,-51.239553,NaN,-3.962362,NaN,NaN,-34.912997,NaN,27.965636,...,127.400216,142.742306,NaN,21.590656,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,12.799991,-40.006417,-43.977176,NaN,-21.475171,NaN,NaN,84.149802,NaN,50.700469,...,-31.230234,-26.123569,NaN,-34.421060,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,-49.265727,-50.818894,-58.748362,-54.705631,-65.864002,-32.443820,-60.649983,-35.369814,-58.487234,-69.452376,...,-61.709418,-62.066094,-26.076055,-64.235865,-39.078995,-65.669996,-72.241465,NaN,NaN,NaN
2019-12-26,61.533841,92.086438,581.425167,54.651605,103.678970,75.690526,92.086331,-28.771615,116.324507,135.418624,...,62.925083,92.735824,-5.897871,82.349374,28.010158,71.569990,567.825312,NaN,NaN,NaN
2019-12-27,26.804677,57.070570,3.017284,9.983210,16.206117,-1.901783,55.565543,13.691915,4.040481,9.066465,...,1.161331,9.872560,6.397677,15.229746,48.337029,24.810203,-45.455759,NaN,NaN,NaN


In [150]:
today = dt.date(2000, 1, 15)
simend = dt.date(2019, 12, 31)
tickers = []
transactionid = 0
money = 1000000
portfolio = {}
activelog = []
transactionlog = []

In [151]:
prices

,MSFT,AAPL,AMZN,FB,JPM,GOOGL,GOOG,JNJ,V,PG,...,ADS,JWN,LB,UAA,GPS,UA,COTY,NWS,WCG,ARNC
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,37.199776,0.863657,89.375000,NaN,26.053429,NaN,NaN,27.165655,NaN,30.795565,...,NaN,7.681504,7.265524,NaN,27.613022,NaN,NaN,NaN,NaN,NaN
2000-01-04,35.943153,0.790842,81.937500,NaN,25.481777,NaN,NaN,26.171118,NaN,30.203011,...,NaN,7.383189,7.119992,NaN,26.612549,NaN,NaN,NaN,NaN,NaN
2000-01-05,36.322121,0.802415,69.750000,NaN,25.324482,NaN,NaN,26.447380,NaN,29.628397,...,NaN,7.252682,7.019238,NaN,25.452013,NaN,NaN,NaN,NaN,NaN
2000-01-06,35.105415,0.732975,65.562500,NaN,25.684019,NaN,NaN,27.276169,NaN,30.993101,...,NaN,7.606922,6.907290,NaN,28.453421,NaN,NaN,NaN,NaN,NaN
2000-01-07,35.564171,0.767695,69.562500,NaN,26.155895,NaN,NaN,28.436464,NaN,33.471111,...,NaN,7.606922,6.952066,NaN,29.814056,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,156.137955,70.580566,1789.209961,205.119995,132.758896,1344.430054,1343.560059,143.054947,186.703415,122.880356,...,109.267487,39.905006,17.997805,21.639999,16.765017,19.400000,10.821499,14.060658,NaN,NaN
2019-12-26,157.417786,71.980911,1868.770020,207.789993,134.167709,1362.469971,1360.400024,142.956924,188.286041,122.880356,...,108.617737,40.625710,18.165731,21.799999,17.040632,19.549999,10.811607,14.296807,NaN,NaN
2019-12-27,157.705505,71.953598,1869.800049,208.100006,134.264221,1354.640015,1351.890015,142.878494,188.514984,123.734093,...,109.188736,40.378895,17.987926,21.450001,16.945591,19.250000,10.851174,14.237770,NaN,NaN


In [152]:
def getprice(date, ticker):
    global prices
    return prices.loc[date][ticker]

print(getprice('2000-01-06', 'AAPL'))

0.73297518491745


In [153]:
def transaction(id, ticker, ammount, price, ordertype, info):
    global transactionid
    if ordertype == 'buy':
        exp_date = today + dt.timedelta(days=14)
        transactionid += 1
        
        data = {'id': transactionid,
                'ticker': ticker,
                'ammount': ammount,
                'price': price,
                'date': today,
                'type': ordertype,
                'exp_date': exp_date,
                'info': info}
        activelog.append(data)
        
    else:
        exp_date = today
        if ordertype == 'sell':
            data = {'id': id,
                    'ticker': ticker,
                    'ammount': ammount,
                    'price': price,
                    'date': today,
                    'type': ordertype,
                    'exp_date': exp_date,
                    'info': info}
   
    transactionlog.append(data)
        
        

In [154]:
def buy(interestedTickers, allocatedMoney):
    global money, portfolio
    for item in interestedTickers:
        price = getprice(today, item)
        if not np.isnan(price):
            quantity = math.floor(allocatedMoney / price)
            money -= quantity * price
            portfolio[item] += quantity
            transaction(0, item, quantity, price, "buy", "")
        

In [155]:
def sell():
    global money, portfolio, today
    itemstoremove = []
    for i in range(len(activelog)):
        log = activelog[i]
        if log["exp_date"] <= today and log['type'] == "buy":
            tickerPrice = getprice(today, log['ticker'])
            if not np.isnan(tickerPrice):
                money += log['ammount'] * tickerPrice
                portfolio[log['ticker']] -= log['ammount']
                transaction(log['id'], log['ticker'], log['ammount'], log['price'], 'sell', log['info'])
                itemstoremove.append(i)
            else:
                log['exp_date'] += dt.timedelta(days=1)
    itemstoremove.reverse()

    for elem in itemstoremove:
        activelog.remove(activelog[elem])

    

In [156]:
def simulation():
    global today, volumechanges, money
    start_date = today - dt.timedelta(days=14)
    series = volumechanges.loc[start_date:today].mean()
    interestlst = series[series >100].index.tolist()
    print(interestlst)
    sell()
    if len(interestlst) > 0:
        moneyToAllocate = currentvalue()/ (2 * len(interestlst))
        buy(interestlst, moneyToAllocate)
    

In [157]:
# Helpers

def getindises():
    global tickers
    f = open('symbols.txt', 'r')
    for line in f:
        tickers.append(line.strip())
    f.close()
    
def tradingday():
    global prices, today
    return np.datetime64(today) in list(prices.index.values)

def currentvalue():
    global money, portfolio, today, prices
    value = money
    for ticker in tickers:
        tickerprice = getprice(today, ticker)
        if not np.isnan(tickerprice):
            value += portfolio[ticker] * tickerprice
            
    return int(value * 100)/100

In [158]:
def main():
    global today
    getindises()
    for ticker in tickers:
        portfolio[ticker] = 0
    while today < simend:
        while not tradingday():
            today += dt.timedelta(days=1)
        
        simulation()
        currentpvalue = currentvalue()
        print(currentpvalue, today)
        today += dt.timedelta(days=7)
    

In [159]:
main()

['APH', 'ANSS', 'HSY', 'ARE', 'UDR', 'ATO', 'ODFL', 'HFC', 'AIV', 'PRGO']
1000000.0 2000-01-18
['IDXX', 'HSY', 'ARE', 'KMX', 'UDR', 'ODFL', 'TSCO', 'HFC', 'PVH', 'AIV', 'PRGO', 'PWR', 'ROL']
1001697.87 2000-01-25
['SYK', 'BLK', 'PLD', 'CTSH', 'SBAC', 'BXP', 'ARE', 'KMX', 'ODFL', 'HFC', 'PVH', 'SLG', 'PRGO', 'PWR']
1002450.88 2000-02-01
['NKE', 'BLK', 'CTSH', 'SBAC', 'BXP', 'ARE', 'LH', 'ODFL', 'TSCO', 'PVH', 'AIV', 'SLG']
1039275.22 2000-02-08
['APH', 'WEC', 'FE', 'BXP', 'ARE', 'LH', 'DRE', 'TSCO', 'NOV', 'AIV', 'FLIR', 'FLS', 'UNM']
1019174.14 2000-02-15
['BLK', 'APH', 'WEC', 'LEN', 'ZBRA', 'DRE', 'ODFL', 'HFC', 'FLIR', 'FLS', 'UNM']
1009809.93 2000-02-22
['BLK', 'REGN', 'VTR', 'LEN', 'ZBRA', 'ATO', 'ODFL', 'CHRW', 'NI', 'ABMD', 'FLIR', 'ROL']
1033396.57 2000-02-29
['PG', 'BLK', 'EIX', 'VTR', 'IFF', 'ATO', 'ODFL', 'CHRW', 'NI', 'ZION', 'HFC', 'FLIR']
972303.42 2000-03-07
['PG', 'VTR', 'EVRG', 'IFF', 'IEX', 'ODFL', 'PKG', 'REG', 'ZION', 'HFC', 'FLIR']
943501.78 2000-03-14
['PG', 'BLK',

1799894.97 2001-08-02
['DHR', 'MNST', 'ARE', 'KMX', 'GRMN', 'DRI', 'LNT', 'ODFL', 'PKG', 'FTI', 'PRGO', 'RL', 'ROL', 'ADS']
1732753.18 2001-08-09
['WEC', 'VTR', 'ARE', 'DRI', 'CTL', 'ODFL', 'GL', 'DVN', 'FTI']
1774524.93 2001-08-16
['WEC', 'WLTW', 'MNST', 'VTR', 'ALGN', 'CTL', 'ODFL', 'GL', 'REG', 'DVN', 'FTI']
1754868.38 2001-08-23
['OKE', 'WLTW', 'MNST', 'VTR', 'ALGN', 'COO', 'ODFL', 'REG', 'ADS']
1731060.03 2001-08-30
['OKE', 'MNST', 'VTR', 'AEE', 'COO', 'EVRG', 'GRMN', 'ODFL', 'ADS']
1724204.38 2001-09-06
['DIS', 'BKNG', 'CB', 'D', 'RTN', 'BLK', 'NOC', 'EQIX', 'AON', 'LHX', 'GD', 'CTSH', 'WLTW', 'LUV', 'MNST', 'VTR', 'CCL', 'ALGN', 'BXP', 'AEE', 'AJG', 'EVRG', 'TIF', 'GRMN', 'HOLX', 'VAR', 'ODFL', 'RE', 'VNO', 'IRM', 'FTI', 'ALK', 'SLG', 'MOS']
1624380.49 2001-09-17
['RTN', 'PLD', 'NOC', 'EQIX', 'AON', 'SRE', 'GD', 'WLTW', 'LUV', 'MNST', 'CCL', 'AJG', 'ODFL', 'VNO', 'REG', 'HSIC', 'SLG', 'MOS', 'AOS', 'ADS']
1526703.58 2001-09-24
['RTN', 'NOC', 'AVB', 'HOLX', 'ODFL', 'VNO', 'REG', 

4153240.04 2003-09-23
['NDAQ', 'HFC', 'ABMD', 'AOS']
4276953.27 2003-09-30
['REGN', 'MNST', 'NDAQ', 'HFC', 'ABMD', 'AOS']
4417520.43 2003-10-07
['REGN', 'MNST', 'NDAQ', 'PWR']
4539841.37 2003-10-14
['ILMN', 'REGN', 'MNST', 'PWR']
4556695.09 2003-10-21
['ANTM', 'RMD', 'STX']
4585611.26 2003-10-28
['ANTM', 'ISRG', 'ILMN', 'MNST', 'RMD', 'STX', 'ZBRA', 'IT', 'AIV']
4470828.71 2003-11-04
['VRTX', 'ILMN', 'MNST', 'WAB', 'ZBRA', 'IT', 'NDAQ', 'DISH', 'AIV']
4333503.99 2003-11-11
['VRTX', 'ILMN', 'TRV', 'MNST', 'NDAQ', 'RJF', 'LKQ', 'HFC', 'AIV']
4457367.06 2003-11-18
['VRTX', 'ILMN', 'TRV', 'OKE', 'MNST', 'NDAQ', 'RJF', 'LKQ', 'HFC']
4555713.71 2003-11-25
['EQIX', 'ILMN', 'OKE', 'CMS', 'CNP', 'ABMD']
4671757.96 2003-12-02
['CERN', 'CMS', 'CNP', 'NDAQ', 'NRG', 'ABMD', 'NWL']
4551555.61 2003-12-09
['NDAQ', 'NRG', 'NWL']
4527216.14 2003-12-16
['NDAQ', 'LKQ', 'VNO', 'NRG', 'NWL']
4610886.98 2003-12-23
['VTR', 'LKQ', 'VNO', 'NRG', 'HFC', 'ABMD']
4667426.96 2003-12-30
['VTR', 'ABC', 'NDAQ', 'ODFL'

9009714.31 2007-02-14
['EFX']
9232437.65 2007-02-21
['EFX']
8857291.12 2007-02-28
['TTWO']
8510359.52 2007-03-07
['TTWO']
8847661.95 2007-03-14
['TTWO', 'HST', 'EXR']
10007403.17 2007-03-21
['ALXN', 'EXR', 'IVZ']
9225911.9 2007-03-28
['IVZ', 'IPGP']
9414605.25 2007-04-04
['COF', 'ROK', 'URI', 'ODFL', 'AIZ', 'IVZ', 'IPGP']
9491027.18 2007-04-11
['ROK', 'STX', 'URI', 'PBCT']
9995271.91 2007-04-18
['ODFL']
10006709.27 2007-04-25
['ALGN']
9539924.44 2007-05-02
['STZ', 'PEAK', 'MHK']
9636610.58 2007-05-09
['STZ', 'PEAK', 'MHK']
9379408.91 2007-05-16
['TDG', 'HOLX', 'MHK', 'ADS']
9488877.99 2007-05-23
['HOLX', 'ADS']
9537650.9 2007-05-30
[]
9913673.98 2007-06-06
[]
9830075.49 2007-06-13
['MGM', 'IVZ']
9830075.49 2007-06-20
['MGM', 'IVZ']
9996426.58 2007-06-27
['TEL', 'DGX']
10342480.09 2007-07-05
['TEL', 'LYV']
10262264.39 2007-07-12
['DGX', 'LKQ', 'LYV', 'AIZ']
10211445.59 2007-07-19
['LKQ', 'AIZ']
9781399.71 2007-07-26
[]
9699589.25 2007-08-02
[]
9949428.89 2007-08-09
[]
9949428.89 2007-08

17441062.39 2011-07-12
['AVY']
17239978.85 2011-07-19
['ECL', 'HCA', 'ABC']
17477380.39 2011-07-26
['ECL', 'HCA', 'TMUS', 'ABC']
16058234.32 2011-08-02
['TMUS']
14347843.37 2011-08-09
[]
16369672.75 2011-08-16
['MKTX']
16031352.32 2011-08-23
['MKTX']
16181150.71 2011-08-30
[]
15427452.91 2011-09-06
[]
15640647.07 2011-09-13
[]
15640647.07 2011-09-20
['FBHS']
15640647.07 2011-09-27
['FBHS']
15014787.91 2011-10-04
['FBHS']
15229935.81 2011-10-11
['KMI', 'TEL']
17670994.2 2011-10-18
['TEL', 'COO']
18105169.32 2011-10-25
['XYL']
17757135.49 2011-11-01
['XYL']
17203271.7 2011-11-08
['COO']
16635304.32 2011-11-15
['COO']
16660402.8 2011-11-22
['IDXX', 'COO', 'INCY']
17009988.53 2011-11-29
[]
17528740.69 2011-12-06
['VMC', 'MLM']
17795582.02 2011-12-13
['REGN', 'DLTR', 'MLM', 'PRGO']
17825286.26 2011-12-20
['REGN', 'DLTR', 'URI', 'PRGO']
18067332.13 2011-12-27
[]
17816421.16 2012-01-03
[]
18758349.23 2012-01-10
['AGN', 'APTV']
18758349.23 2012-01-17
['AGN']
18868442.54 2012-01-24
[]
19748934.

21200792.03 2015-09-08
[]
21545940.44 2015-09-15
['AMCR']
21841517.86 2015-09-22
['AMCR', 'STE']
21841517.86 2015-09-29
[]
21910218.63 2015-10-06
['VRSK']
21996727.53 2015-10-13
['AMCR', 'GRMN']
21893008.19 2015-10-20
['HPE', 'AMCR', 'GRMN']
22134920.17 2015-10-27
['HPE', 'MKTX']
22119539.31 2015-11-03
['HPE', 'AMCR']
22436617.49 2015-11-10
['AMCR']
21750385.27 2015-11-17
['AMCR']
21933334.96 2015-11-24
[]
21933334.96 2015-12-01
[]
21933334.96 2015-12-08
['AMCR', 'NWL']
21933334.96 2015-12-15
['AMCR']
21970610.58 2015-12-22
['AMCR', 'CHD']
21947003.02 2015-12-29
['WLTW', 'AMCR', 'CHD']
21570748.2 2016-01-05
['WLTW', 'AMCR']
20668438.45 2016-01-12
['AMCR']
20082440.31 2016-01-19
[]
20110165.24 2016-01-26
['FRT']
20110165.24 2016-02-02
['FRT']
19866830.59 2016-02-09
[]
20154725.49 2016-02-16
['AMCR']
20053207.36 2016-02-23
['AMCR']
20053207.36 2016-03-01
['APD', 'AWK', 'UDR']
20053207.36 2016-03-08
['APD', 'AMCR', 'UDR']
22258323.84 2016-03-15
['APD', 'INFO', 'AMCR']
22764410.46 2016-03-

47841801.67 2019-12-10
['ODFL', 'WRB']
47728353.05 2019-12-17
['ZBRA', 'STE']
47501683.49 2019-12-24
